In [127]:
import pandas as pd

In [128]:
ResOCTWS = pd.read_csv('./res/OCTWS.csv')
ResOCTNoWS = pd.read_csv('./res/OCTNoWS.csv')
ResCART = pd.read_csv('./res/CART.csv')
ResOCT_b = pd.read_csv('./res/OCT_b.csv')
ResOCT_b_scale = pd.read_csv('./res/OCT_b_scale.csv')
res = {'OCTWS':ResOCTWS, 'OCTNoWS':ResOCTNoWS, 'OCT_b':ResOCT_b, 'OCT_b_scale':ResOCT_b_scale}

In [129]:
def resProcess(res):
    """
    Calculate the average results from random data split
    Select alpha with highest validation accuracy
    """
    if 'alpha' in res.columns:
        # Calculate the mean of the feat for each instance, depth and alpha
        res_group = res.groupby(['instance', 'depth', 'alpha']).mean().reset_index()
        # Find the max/min value of the feat for each instance and depth
        res_group = res_group.groupby(['instance', 'depth']) \
                             .apply(lambda group: group[group['train_time'] == group['train_time'].min()]) \
                             .drop_duplicates(subset=['instance', 'depth']) \
                             .reset_index(drop=True)
    else:
        res_group = res.groupby(['instance', 'depth']).mean().reset_index()
    return res_group

In [130]:
def resCombine(res, feat):
    df = pd.DataFrame(columns=['instance', 'depth'])
    for r in res:
        df_cur = resProcess(res[r])[['instance', 'depth', feat]]
        df_cur = df_cur.rename(columns={feat:r})
        df = df.merge(df_cur, how='outer', on=['instance', 'depth'])
    return df

In [131]:
resCombine(res, 'gap').to_csv('./res/res.csv')